In [ ]:
from kafka import KafkaConsumer
import json
import time
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassificationModel
from pyspark.sql import Row

# ✅ 1. Spark Session
spark = SparkSession.builder \
    .appName("KafkaConsumerDT_IgnoreErrors") \
    .getOrCreate()

# ✅ 2. Charger le modèle Decision Tree entraîné
model_path = "models/decision_tree_model"  # adapte ce chemin si besoin
dt_model = DecisionTreeClassificationModel.load(model_path)

# ✅ 3. Kafka Consumer (lit depuis le début du topic)
consumer = KafkaConsumer(
    'accidents-stream',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',  # ← lire depuis le début
    enable_auto_commit=True,
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

print("✅ Consumer démarré. Messages valides uniquement.\n")

# ✅ 4. Boucle de consommation
for msg in consumer:
    data = msg.value
    features_data = data.get("features", {}).get("values", [])

    try:
        # Créer vecteur Spark à partir des features
        features_vector = Vectors.dense(features_data)
        row = Row(features=features_vector)
        input_df = spark.createDataFrame([row])

        # Appliquer la prédiction
        prediction_df = dt_model.transform(input_df)

        # Afficher uniquement les résultats valides
        prediction_df.select("features", "prediction", "probability").show(truncate=False)

    except Exception as e:
        # 🔕 Ignorer les erreurs silencieusement
        print("⚠️ Message ignoré (erreur):", e)
        continue

    time.sleep(0.2)  # ⏱ ralentir un peu pour la lecture continue


✅ Consumer démarré. Messages valides uniquement.



25/07/08 04:21:41 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
25/07/08 04:21:42 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
                                                                                

+---------------------------------------------------------------------+----------+--------------------------------------------------------------------------------------+
|features                                                             |prediction|probability                                                                           |
+---------------------------------------------------------------------+----------+--------------------------------------------------------------------------------------+
|[92.0,92.0,48.0,29.58,10.0,5.0,0.0,0.364,230.0,266.0,7.0,9.0,3.0,0.0]|2.0       |[0.0,0.0015723627704486687,0.929899583884342,0.034984314242304436,0.03354373910290493]|
+---------------------------------------------------------------------+----------+--------------------------------------------------------------------------------------+



25/07/08 04:21:43 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
25/07/08 04:21:44 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
                                                                                

+------------------------------------------------------------------+----------+---------------------------------------------------------------------------------------+
|features                                                          |prediction|probability                                                                            |
+------------------------------------------------------------------+----------+---------------------------------------------------------------------------------------+
|[75.0,75.0,96.0,29.96,8.0,5.0,0.16,6.072,0.0,1.0,1.0,17.0,5.0,1.0]|2.0       |[0.0,0.0017803557358636066,0.9467737338854336,0.03281420261185898,0.018631707766843808]|
+------------------------------------------------------------------+----------+---------------------------------------------------------------------------------------+



...

25/07/08 04:24:33 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
25/07/08 04:24:34 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB


+--------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------+
|features                                                            |prediction|probability                                                                             |
+--------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------+
|[67.0,67.0,81.0,29.78,7.0,3.0,0.0,0.175,1066.0,78.0,0.0,2.0,2.0,1.0]|2.0       |[0.0,4.815939944956799E-4,0.990600754497258,0.0033425888262030665,0.0055750626820432105]|
+--------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------+



25/07/08 04:24:35 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
25/07/08 04:24:35 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
                                                                                

+-----------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+
|features                                                               |prediction|probability                                                                          |
+-----------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+
|[76.0,76.0,22.0,24.42,10.0,7.0,0.0,0.179,7384.0,432.0,13.0,4.0,0.0,1.0]|2.0       |[0.0,0.006246096189881324,0.5484072454715803,0.16489693941286696,0.28044971892567144]|
+-----------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+



25/07/08 04:24:37 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
25/07/08 04:24:37 WARN DAGScheduler: Broadcasting large task binary with size 1309.8 KiB
[Stage 234:>                                                        (0 + 3) / 3]